In [12]:
import voice_to_text as vtt
import Mel_creator as mc
from data_utils_2 import TextMelLoader, TextMelCollate
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader

In [13]:

inpath ='C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test'
import os
os.listdir(inpath)

['LJ001-0001.wav',
 'LJ001-0002.wav',
 'LJ001-0003.wav',
 'LJ001-0004.wav',
 'LJ001-0005.wav']

In [14]:

inpath ='C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test'
outpath = 'C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE'

vtt.voice_to_text(inpath,outpath,language="english")


Starting...
LJ001-0001.wav
C:\Users\derec\OneDrive\Documents\F-VOICE\audio_test\LJ001-0001.wav
LJ001-0002.wav
C:\Users\derec\OneDrive\Documents\F-VOICE\audio_test\LJ001-0002.wav
LJ001-0003.wav
C:\Users\derec\OneDrive\Documents\F-VOICE\audio_test\LJ001-0003.wav
LJ001-0004.wav
C:\Users\derec\OneDrive\Documents\F-VOICE\audio_test\LJ001-0004.wav
LJ001-0005.wav
C:\Users\derec\OneDrive\Documents\F-VOICE\audio_test\LJ001-0005.wav


In [15]:
input_path = 'C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test'
output_path = 'C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\mel_audio'

mc.mel_creator(input_path,output_path)

Mels state: 100%|██████████| 5/5 [00:03<00:00,  1.50it/s]

wavs_mel created


In [16]:

hparams = {
    'training_files':'C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\wavs_text.txt',
    'validation_files':'C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\wavs_text.txt',
    'text_cleaners': ['english_cleaners'],
    'max_wav_value': 32768.0,
    'sampling_rate': 22050,
    'load_mel_from_disk': False,
    'filter_length': 1024,
    'hop_length': 256,
    'win_length': 1024,
    'n_mel_channels': 80,
    'mel_fmin': 0.0,
    'mel_fmax': 8000.0,
    'seed': 20,
    'distributed_run':False,
     ################################
        # Optimization Hyperparameters #
        ################################
    'use_saved_learning_rate':False,
    'learning_rate':1e-3,
    'weight_decay':1e-6,
    'grad_clip_thresh':1.0,
    'batch_size':64,
    'mask_padding':True  # set model's padded outputs to padded values
}


def prepare_dataloaders(hparams):
    # Get data, data loaders and collate function ready
    trainset = TextMelLoader(hparams['training_files'], hparams['text_cleaners'],
                         hparams['max_wav_value'], hparams['sampling_rate'],
                         hparams['load_mel_from_disk'], hparams['filter_length'],
                         hparams['hop_length'], hparams['win_length'],
                         hparams['n_mel_channels'], hparams['mel_fmin'],
                         hparams['mel_fmax'], hparams['seed'])
    valset = TextMelLoader( hparams['validation_files'], hparams['text_cleaners'],
                         hparams['max_wav_value'], hparams['sampling_rate'],
                         hparams['load_mel_from_disk'], hparams['filter_length'],
                         hparams['hop_length'], hparams['win_length'],
                         hparams['n_mel_channels'], hparams['mel_fmin'],
                         hparams['mel_fmax'], hparams['seed'])
    collate_fn = TextMelCollate(n_frames_per_step=1)

    if hparams['distributed_run']:
        train_sampler = DistributedSampler(trainset)
        shuffle = False
    else:
        train_sampler = None
        shuffle = True

    train_loader = DataLoader(trainset, num_workers=1, shuffle=shuffle,
                              sampler=train_sampler,
                              batch_size=hparams['batch_size'], pin_memory=False,
                              drop_last=True, collate_fn=collate_fn)
    return train_loader, valset, collate_fn

In [17]:
t,v,c =prepare_dataloaders(hparams)
print(v.audiopaths_and_text)
v.get_mel_text_pair(v.audiopaths_and_text[0])

[['C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test/LJ001-0005.wav ', ' The invention of movable metal letters in the middle of the 15th century may justly be considered, as the invention of the art of printing.'], ['C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test/LJ001-0001.wav ', ' Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the exhibition.'], ['C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test/LJ001-0004.wav ', ' produced the block books, which were the immediate predecessors of the true printed book'], ['C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test/LJ001-0003.wav ', ' For although the Chinese took impressions from wood blocks and graved in relief for centuries before the wood cutters of the Netherlands by a similar process,'], ['C:\\Users\\derec\\OneDrive\\Documents\\F-VOICE\\audio_test/LJ001-0002.wav ', ' in being comparatively modern.']]


(tensor([11, 57, 45, 42, 11, 46, 51, 59, 42, 51, 57, 46, 52, 51, 11, 52, 43, 11,
         50, 52, 59, 38, 39, 49, 42, 11, 50, 42, 57, 38, 49, 11, 49, 42, 57, 57,
         42, 55, 56, 11, 46, 51, 11, 57, 45, 42, 11, 50, 46, 41, 41, 49, 42, 11,
         52, 43, 11, 57, 45, 42, 11, 43, 46, 43, 57, 42, 42, 51, 57, 45, 11, 40,
         42, 51, 57, 58, 55, 62, 11, 50, 38, 62, 11, 47, 58, 56, 57, 49, 62, 11,
         39, 42, 11, 40, 52, 51, 56, 46, 41, 42, 55, 42, 41,  6, 11, 38, 56, 11,
         57, 45, 42, 11, 46, 51, 59, 42, 51, 57, 46, 52, 51, 11, 52, 43, 11, 57,
         45, 42, 11, 38, 55, 57, 11, 52, 43, 11, 53, 55, 46, 51, 57, 46, 51, 44,
          7], dtype=torch.int32),
 tensor([[-9.1835, -6.4719, -6.0837,  ..., -7.4548, -7.7138, -7.3676],
         [-6.7287, -6.1699, -6.1468,  ..., -6.8450, -7.2991, -6.9986],
         [-5.8363, -5.4046, -5.2813,  ..., -6.3080, -6.4373, -6.1320],
         ...,
         [-6.1044, -5.9024, -5.3051,  ..., -9.3646, -9.5347, -9.8833],
         [-5.6057, -